In [74]:
import os

import tiktoken
import fitz

from qdrant_client import QdrantClient
from dotenv import load_dotenv
from openai import OpenAI

In [75]:
def get_qdrant_client() -> QdrantClient:

    load_dotenv()

    # IP und Port aus Umgebungsvariablen lesen
    host = os.getenv("QDRANT_SERVER_IP", "localhost")
    port = int(os.getenv("QDRANT_PORT", 6333))

    try:
        client = QdrantClient(host=host, port=port)

        # Verbindung testen (z. B. durch Auflisten der Collections)
        _ = client.get_collections()

        print(f"Verbindung zu Qdrant unter {host}:{port} erfolgreich!")
        return client

    except Exception as e:
        print(f"Fehler bei der Verbindung zu Qdrant ({host}:{port}): {e}")
        return None

In [76]:
def is_qdrant_alive(client) -> bool:
    """
    Prüft, ob Qdrant erreichbar ist, indem eine einfache Anfrage gestellt wird.
    """
    try:
        client.get_collections()
        return True
    except Exception as e:
        print(f"Verbindungsprüfung fehlgeschlagen: {e}")
        return False



In [77]:
def load_pdf_and_chunk(filepath: str, chunk_size: int = 500, overlap: int = 50) -> list[str]:
    # Existenz der Datei prüfen
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Die Datei wurde nicht gefunden: {filepath}")

    # PDF einlesen
    doc = fitz.open(filepath)
    full_text = ""
    for page in doc:
        full_text += page.get_text("text") + "\n"  # "\n" trennt die Seiten
    doc.close()

    # Chunks erzeugen mit Überlappung
    chunks = []
    start = 0
    while start < len(full_text):
        end = min(start + chunk_size, len(full_text))
        chunks.append(full_text[start:end])
        start += chunk_size - overlap

    return chunks

In [78]:
def get_embedding(text: str, model: str = "text-embedding-3-large") -> list:
    """
    Erstellt ein Embedding für einen gegebenen Text über die OpenAI API.

    Args:
        text (str): Der zu embeddene Text
        model (str): Das OpenAI-Embedding-Modell (default: text-embedding-3-large)

    Returns:
        list: Embedding-Vektor als Liste von Floats
    """
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")

    if not api_key:
        raise ValueError("OPENAI_API_KEY wurde nicht in der .env-Datei gefunden.")

    client = OpenAI(api_key=api_key)

    try:
        response = client.embeddings.create(
            input=text,
            model=model
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Fehler beim Abrufen des Embeddings: {e}")
        return []

In [79]:
def embed_chunks(chunks: list[str]) -> list[list[float]]:
    return [get_embedding(chunk) for chunk in chunks]


In [80]:
from qdrant_client.models import PointStruct, VectorParams, Distance

def store_embeddings_in_qdrant(client, collection_name: str, chunks: list[str], embeddings: list[list[float]]):
    # Collection neu erstellen (löscht alte Inhalte!)
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE)
    )

    # Punkte erstellen
    points = [
        PointStruct(id=i, vector=vector, payload={"text": chunks[i]})
        for i, vector in enumerate(embeddings)
    ]

    # Hochladen
    client.upsert(collection_name=collection_name, points=points)

    print(f"{len(points)} Embeddings in Qdrant gespeichert.")


In [81]:
def retrieve_similar_chunks(query: str, client: QdrantClient, collection_name: str, top_k: int = 5) -> list[str]:
    """
    Führt eine semantische Suche in Qdrant basierend auf einer Query durch.

    Args:
        query (str): Die Nutzerfrage
        client (QdrantClient): Verbundener Qdrant-Client
        collection_name (str): Name der zu durchsuchenden Collection
        top_k (int): Anzahl der zurückgegebenen ähnlichen Chunks

    Returns:
        list[str]: Liste der ähnlichsten Text-Chunks
    """
    query_vector = get_embedding(query)
    if not query_vector:
        return []

    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=top_k
    )

    return [hit.payload["text"] for hit in search_result]


In [82]:
def answer_with_context(query: str, context_chunks: list[str], model: str = "gpt-4o") -> str:
    """
    Nutzt OpenAI GPT-4o, um eine Antwort auf eine Frage zu geben – basierend auf den gegebenen Kontext-Chunks.

    Args:
        query (str): Die Benutzerfrage
        context_chunks (list[str]): Liste von Texten aus Qdrant
        model (str): OpenAI-Modellname (standardmäßig GPT-4o)

    Returns:
        str: Generierte Antwort
    """
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY wurde nicht in der .env-Datei gefunden.")

    client = OpenAI(api_key=api_key)

    # Prompt zusammenbauen
    context = "\n\n".join(context_chunks)
    prompt = f"Beantworte die folgende Frage basierend auf dem Kontext:\n\nKontext:\n{context}\n\nFrage: {query}"

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Du bist ein hilfreicher Assistent für wissenschaftliche Fragen."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Fehler bei der Antwortgenerierung: {e}")
        return "Fehler bei der Antwortgenerierung."


In [83]:
if __name__ == "__main__":
    client = get_qdrant_client()
    is_qdrant_alive(client)

    pdf_path = "/Users/i589466/Desktop/Datenbanken/Datenbanken/Konz_Julian_Abstract_PA2.pdf"
    chunks = load_pdf_and_chunk(pdf_path, chunk_size=500, overlap=50)
    embeddings = embed_chunks(chunks)
    store_embeddings_in_qdrant(client, "db_Benny", chunks, embeddings)

    if is_qdrant_alive(client):
        query= "Wann wurde der Abstract geschrieben?"
        retrieved_chunks = retrieve_similar_chunks(query, client, "db_Benny", top_k=5)
        answer = answer_with_context(query, retrieved_chunks)
        print(f"\n🧠 Antwort von GPT-4o:\n{answer}")

Verbindung zu Qdrant unter 152.53.228.53:6333 erfolgreich!


/var/folders/06/y765zhbd3xg97g0s0prnz96r0000gn/T/ipykernel_80466/1369516826.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


4 Embeddings in Qdrant gespeichert.


/var/folders/06/y765zhbd3xg97g0s0prnz96r0000gn/T/ipykernel_80466/2305006633.py:18: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(



🧠 Antwort von GPT-4o:
Der Abstract enthält keine spezifischen Informationen über das Datum, an dem er geschrieben wurde. Um das genaue Datum zu bestimmen, wäre es notwendig, auf zusätzliche Informationen wie das Veröffentlichungsdatum der Arbeit oder das Datum der Einreichung zuzugreifen, die in dem bereitgestellten Kontext nicht enthalten sind.
